# Cryptography Basics

This notebook demonstrates the basics of cryptography using Python's cryptography library alongside OpenSSL commands.

We'll cover:
1. PKI (Public Key Infrastructure)
2. Key Pairs
3. Certificate Signing Requests (CSR)
4. X.509 Certificates

First, let's install and import required packages:

In [ ]:
%pip install cryptography

In [5]:
import os
from cryptography import x509
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import serialization
from datetime import datetime, timedelta

Lets have some code which loads the key, csr, and certificate.


In [6]:
def load_cert(cert: bytes) -> x509.Certificate:
    try:
        return x509.load_pem_x509_certificate(cert)
    except:
        pass  # fall back to DER

    return x509.load_der_x509_certificate(cert)

def load_key(key: bytes):
    try:
        return serialization.load_pem_private_key(key, password=None)
    except:
        pass  # fall back to DER

    return serialization.load_der_private_key(key, password=None)

def load_csr(csr: bytes) -> x509.CertificateSigningRequest:
    try:
        return x509.load_pem_x509_csr(csr)
    except:
        pass  # fall back to DER

    return x509.load_der_x509_csr(csr)

def load_crl(crl: bytes) -> x509.CertificateRevocationList:
    try:
        return x509.load_pem_x509_crl(crl)
    except:
        pass  # fall back to DER

    return x509.load_der_x509_crl(crl)

#### Key Pair Generation

A key pair consists of a public key and a private key. The private key must be kept secret while the public key can be freely shared.

In [7]:
def generate_ec_secp256r1_keypair() -> bytes:
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    return private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption(),
    )

keypair = generate_ec_secp256r1_keypair()

with open('keypair.pem', 'wb') as f:
    f.write(keypair)

#### Certificate Signing Request (CSR)

A CSR is a message sent from an applicant to a CA to apply for a digital certificate.

In [8]:
def generate_csr(keypair: bytes, cn: str) -> bytes:

    builder = x509.CertificateSigningRequestBuilder()
    subject = x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, cn)])
    builder = builder.subject_name(subject)

    keypair_obj = load_key(keypair)
    csr = builder.sign(keypair_obj, hashes.SHA256())

    return csr.public_bytes(serialization.Encoding.PEM)

csr = generate_csr(keypair, 'leaf cert')

with open('csr.pem', 'wb') as f:
    f.write(csr)

#### Self-Signed Certificate (Root CA)

Let's create a self-signed certificate that can act as a root CA:

In [ ]:
def generate_self_signed_cert(keypair: bytes, cn: str) -> bytes:
    keypair_obj = load_key(keypair)
    subject = issuer = x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, cn)])

    cert = x509.CertificateBuilder()
    cert = cert.subject_name(subject)
    cert = cert.issuer_name(issuer)
    cert = cert.public_key(keypair_obj.public_key())
    cert = cert.serial_number(x509.random_serial_number())
    cert = cert.not_valid_before(datetime.utcnow())
    cert = cert.not_valid_after(datetime.utcnow() + timedelta(days=365)) # a year long validity

    cert = cert.add_extension(x509.BasicConstraints(ca=True, path_length=None), critical=True)
    cert = cert.add_extension(x509.SubjectKeyIdentifier.from_public_key(keypair_obj.public_key()), critical=False)
    cert = cert.add_extension(x509.AuthorityKeyIdentifier.from_issuer_public_key(keypair_obj.public_key()), critical=False)
    cert = cert.add_extension(x509.KeyUsage(digital_signature=True, content_commitment=False,
                                            key_encipherment=False, data_encipherment=False,
                                            key_agreement=False, key_cert_sign=True, crl_sign=True,
                                            encipher_only=False, decipher_only=False), critical=True)
    
    cert = cert.sign(keypair_obj, hashes.SHA256())

    return cert.public_bytes(serialization.Encoding.PEM)

ca_key = generate_ec_secp256r1_keypair()
ca_cert = generate_self_signed_cert(ca_key, "I'm G-root")

with open('ca_cert.pem', 'wb') as f:
    f.write(ca_cert)

with open('ca_key.pem', 'wb') as f:
    f.write(ca_key)

#### Issue a Certificate

In [ ]:
def issue_certificate(ca_cert: bytes, ca_key: bytes, csr: bytes, days=365):
    keypair_obj = load_key(ca_key)
    ca_cert_obj = load_cert(ca_cert)
    csr_obj = load_csr(csr)

    cert = x509.CertificateBuilder()
    cert = cert.subject_name(csr_obj.subject)
    cert = cert.issuer_name(ca_cert_obj.subject)
    cert = cert.public_key(csr_obj.public_key())
    cert = cert.serial_number(x509.random_serial_number())
    cert = cert.not_valid_before(datetime.utcnow())
    cert = cert.not_valid_after(datetime.utcnow() + timedelta(days=days))

    cert = cert.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True)
    cert = cert.add_extension(x509.SubjectKeyIdentifier.from_public_key(csr_obj.public_key()), critical=False)
    cert = cert.add_extension(x509.AuthorityKeyIdentifier.from_issuer_public_key(ca_cert_obj.public_key()), critical=False)
    cert = cert.add_extension(x509.KeyUsage(digital_signature=True, content_commitment=False,
                                            key_encipherment=False, data_encipherment=False,
                                            key_agreement=False, key_cert_sign=False, crl_sign=False,
                                            encipher_only=False, decipher_only=False), critical=True)
    cert = cert.sign(keypair_obj, hashes.SHA256())

    return cert.public_bytes(serialization.Encoding.PEM)

leaf_cert = issue_certificate(ca_cert, ca_key, csr)

with open('leaf_cert.pem', 'wb') as f:
    f.write(leaf_cert)

#### Verify Certificate Chain

Let's verify that our leaf certificate was indeed signed by our CA:

In [ ]:
def verify_cert_chain(cert, ca_cert):
    cert_obj = load_cert(cert)
    ca_cert_obj = load_cert(ca_cert)

    cert_akid = cert_obj.extensions.get_extension_for_oid(x509.OID_AUTHORITY_KEY_IDENTIFIER).value.key_identifier
    root_skid = ca_cert_obj.extensions.get_extension_for_oid(x509.OID_SUBJECT_KEY_IDENTIFIER).value.key_identifier
    if cert_akid is None or root_skid is None or cert_akid != root_skid:
        return False

    if cert_obj.issuer != ca_cert_obj.subject:
        return False

    try:
        ca_cert_obj.public_key().verify(cert_obj.signature, cert_obj.tbs_certificate_bytes, ec.ECDSA(cert_obj.signature_hash_algorithm))
    except Exception as e:
        print(e)
        print(f"Signature verification failed for cert subject: {cert_obj.subject}, issuer: {ca_cert_obj.subject}")
        return False

    return True

verify_cert_chain(leaf_cert, ca_cert)